<a href="https://colab.research.google.com/github/FoxPowerGH/GenAI-Journey/blob/main/intro_gemini_2_0_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuración del Ambiente - Google CoLab**

## **Instalamos Google Gen AI SDK**

In [8]:
# Instalo las librerías requeridas en CoLab
%pip install --upgrade --quiet google-genai


In [2]:
# Reiniciamos el kernel luego de la instalación para que el ambiente pueda acceder a los nuevos paquetes
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [7]:
#Cargamos las dependencias requeridas
import os

try:
    from google.colab import drive, userdata, auth
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Autenticación & Secrets
if COLAB:
    auth.authenticate_user()
    GOOGLE_CLOUD_PROJECT = os.environ["GOOGLE_CLOUD_PROJECT"] = userdata.get('GOOGLE_CLOUD_PROJECT')
    GOOGLE_CLOUD_REGION  = os.environ["GOOGLE_CLOUD_REGION"] = userdata.get('GOOGLE_CLOUD_REGION')
    os.environ["SYSTEM_INSTRUCTION"] = userdata.get('SYSTEM_INSTRUCTION')

Note: using Google CoLab


## **Configuramos nuestro entorno de Vertex AI**

In [8]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)

In [13]:
client = genai.Client(vertexai=True, project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

## **Escogemos el Modelo a utilizar**

In [15]:
#Cargamos uno por defecto - Gemini 2.0 Flash model
MODEL_ID = "gemini-2.0-flash-exp"  # @param {type: "string"}

# **Ejemplos prácticos**

### **Generar texto a partir del texto de un prompt simple**

In [17]:
# Realizamos una prueba de conectividad
prompt = "¿Cuál es el planeta más grande en nuestro sistema solar?"

response = client.models.generate_content(
    model=MODEL_ID, contents=prompt
)

display(Markdown(response.text))

El planeta más grande de nuestro sistema solar es **Júpiter**.


## **Iniciar un chat de múltiples turnos**

In [20]:
chat = client.chats.create(model=MODEL_ID)

In [23]:
prompt = "Genera una pregunta de selección múltiple hacerca de fundamentos de SOC L1"

response = chat.send_message(prompt)

display(Markdown(response.text))

¿Cuál de las siguientes tareas **NO** es típicamente una responsabilidad de un analista SOC L1?

a) Monitorear alertas de seguridad y eventos de seguridad.
b) Realizar triaje inicial de alertas y determinar su prioridad.
c) Desarrollar y ajustar reglas de detección de amenazas.
d) Escalar incidentes a equipos de nivel superior (L2/L3) cuando sea necesario.


In [24]:
response = chat.send_message("la respuesta es b")

display(Markdown(response.text))

En realidad, la respuesta correcta es **c) Desarrollar y ajustar reglas de detección de amenazas.**

Aquí está la explicación:

*   **a) Monitorear alertas de seguridad y eventos de seguridad:** Esta es una tarea fundamental del L1.
*   **b) Realizar triaje inicial de alertas y determinar su prioridad:** El L1 necesita evaluar rápidamente las alertas para determinar cuáles son las más urgentes.
*   **d) Escalar incidentes a equipos de nivel superior (L2/L3) cuando sea necesario:** Cuando un incidente es complejo o requiere un conocimiento más profundo, el L1 lo escala.
*   **c) Desarrollar y ajustar reglas de detección de amenazas:** Esta tarea generalmente requiere un conocimiento más profundo del sistema, habilidades de análisis de amenazas y, por lo tanto, es más común para los analistas L2 o L3. Los analistas L1 generalmente trabajan con reglas de detección predefinidas y configuraciones estándar.

Espero que esto aclare la respuesta.


## **Configurar parámetros del model**

In [26]:
prompt = "Explícame los peligros del Internet, para mis hijos menores de edad."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        temperature=0.4,            # Baja creatividad
        top_k=20,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,      # Tengo pocos token para responder
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
    ),
)

display(Markdown(response.text))

¡Claro que sí! Explicar los peligros de internet a tus hijos es fundamental para que puedan disfrutar de la red de forma segura. Aquí te presento una explicación adaptada para menores de edad, enfocándonos en los riesgos más comunes y cómo prevenirlos:

**Introducción: Internet es como una ciudad grande**

"Imaginen que internet es como una ciudad enorme, llena de cosas increíbles para ver y hacer. Puedes aprender, jugar, hablar con amigos y descubrir cosas nuevas

## **Establecer instrucciones del sistema**

In [29]:
system_instruction = """
  Eres un asistente especializado en traducción.
  Tu misión es trauducir texto de español a inglés.
"""

prompt = "Me gustan las hamburguesas."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

I like hamburgers.


In [30]:
prompt = "Favor traducir al frances el siguiente texto:   Me gustan los bagels."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

La traducción de "Me gustan los bagels" al francés es:

**J'aime les bagels.**


In [31]:
system_instruction = """
  Eres un asistente especializado en traducción.
  Tu misión es trauducir texto de español a inglés.
  Si te solicitan traducir a otro idioma, responde de forma educada y amigable tu misión.
"""

prompt = "Favor traducir al frances el siguiente texto:   Me gustan los bagels."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

Soy un asistente especializado en traducción de español a inglés. No puedo traducir a francés.


## **Filtros de seguridad**

Cuando haces una solicitud a Gemini, el contenido es analizado y se le asigna una calificación de seguridad. Podemos inspeccionar las calificaciones de seguridad del contenido generado imprimiendo las respuestas del modelo.

Los ajustes de seguridad están **DESACTIVADOS** por defecto, y los umbrales predeterminados de bloqueo están configurados en **BLOCK_NONE**.

Puedes utilizar safety_settings para ajustar la configuración de seguridad en cada solicitud que realices a la API.

Este ejemplo muestra cómo configurar el umbral de bloqueo en BLOCK_LOW_AND_ABOVE para todas las categorías:

In [32]:

prompt = """
    Escribe una lista de 5 cosas odiosas, crueles y irrespetuosas que podría
    decirle al universo después de golpearme el dedo del pie en la oscuridad
"""

safety_settings = [
    SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        safety_settings=safety_settings,
    ),
)

# Response will be `None` if it is blocked.
print(response.text)
print(response.candidates[0].finish_reason)

for safety_rating in response.candidates[0].safety_ratings:
    print(safety_rating)


None
FinishReason.SAFETY
blocked=None category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=3.966169e-05 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=None
blocked=None category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=7.5322026e-07 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=None
blocked=True category=<HarmCategory.HARM_CATEGORY_HARASSMENT: 'HARM_CATEGORY_HARASSMENT'> probability=<HarmProbability.MEDIUM: 'MEDIUM'> probability_score=0.2674951 severity=<HarmSeverity.HARM_SEVERITY_MEDIUM: 'HARM_SEVERITY_MEDIUM'> severity_score=0.2343337
blocked=None category=<HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 'HARM_CATEGORY_SEXUALLY_EXPLICIT'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGI